## MSBX 5420 Assignment 4
This assignment includes two parts: (1) Graph analysis with Spark GraphFrames (Task 1 and 2); (2) Load data from employees data and do a simple analysis (Task 3). Three sets of data are used in the assignment: facebook social networks (Task 1), reddit community links (Task 2), and two data files related to employees in an organization (Task 3).

### Task 1 - Graph Analysis on Facebook Networks

The data is from Facebook circles. For social networks, the data sometimes looks simple but boring - to protect privacy, only (recoded) user id is available and each row in the data is the connection or friendship from one user to another. 

Let's first load graphframes package and build the graph.

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[4]') \
                    .config("spark.driver.memory", "2g") \
                    .config("spark.jars", "graphframes-0.8.3-spark3.5-s_2.12.jar") \
                    .config("spark.packages", "graphframes:graphframes:0.8.3-spark3.5-s_2.12") \
                    .appName('spark_graph').getOrCreate()

In [4]:
#make sure graphframes-0.8.3-spark3.5-s_2.12.jar is under same directory
sc = spark.sparkContext
sc.addPyFile('graphframes-0.8.3-spark3.5-s_2.12.jar')

In [5]:
#first read the dataset
import pyspark.sql.functions as fn

#this is a txt file without header so after reading data we use .toDF() to add column names
fb_connection = spark.read.csv('./facebook_combined.txt.gz', sep=' ').toDF('from', 'to')
fb_connection.show()

+----+---+
|from| to|
+----+---+
|   0|  1|
|   0|  2|
|   0|  3|
|   0|  4|
|   0|  5|
|   0|  6|
|   0|  7|
|   0|  8|
|   0|  9|
|   0| 10|
|   0| 11|
|   0| 12|
|   0| 13|
|   0| 14|
|   0| 15|
|   0| 16|
|   0| 17|
|   0| 18|
|   0| 19|
|   0| 20|
+----+---+
only showing top 20 rows



Create vertices and edges dataframes, with `id` for vertices, and `src` / `dst` for edges.

In [6]:
#create vertices dataframe
fb_vertices = fb_connection.select(fn.col('from').alias('id')).union(fb_connection.select(fn.col('to').alias('id'))).distinct()
fb_vertices.count()

4039

Because Graphframes by default uses multi-directed graph and there is no "undirected" definition, we need to "duplicate" the edges to have two edges between two nodes to capture their friend relationship on Facebook.

In [7]:
#create edges dataframe
fb_edges = fb_connection.union(fb_connection.select(fn.col('to').alias('from'),fn.col('from').alias('to'))) \
                        .withColumnRenamed('from', 'src').withColumnRenamed('to', 'dst').distinct()
fb_edges.show()
fb_edges.count()

+---+----+
|src| dst|
+---+----+
|  0| 258|
|  0| 294|
|  9| 134|
| 13| 332|
| 40| 200|
| 81| 286|
| 87| 291|
|107|1161|
|107|1452|
|107|1522|
|107|1541|
|107|1640|
|107|1829|
|141| 258|
|180| 302|
|186| 325|
|198| 351|
|251| 281|
|257| 344|
|348| 350|
+---+----+
only showing top 20 rows



176468

In total the data contains 4,039 users and 176,468 edges (bi-directional friendship), consistent with the data description. Then we can build the graph with the two dataframes.

In [8]:
from graphframes import *
#build graph
fb_graph = GraphFrame(fb_vertices, fb_edges)
print(fb_graph)

GraphFrame(v:[id: string], e:[src: string, dst: string])


Let's first get degree centrality. Because friendship tie in Facebook is essentially undirected (bi-directional in our data setup), inDegree and outDegree are actually same here.

In [9]:
#because this is an undirected graph (Facebook only has friendship, not following / followed), inDegree and outDegree are same here
fb_graph.inDegrees.sort(fn.desc("inDegree")).show()
fb_graph.outDegrees.sort(fn.desc("outDegree")).show()

+----+--------+
|  id|inDegree|
+----+--------+
| 107|    1045|
|1684|     792|
|1912|     755|
|3437|     547|
|   0|     347|
|2543|     294|
|2347|     291|
|1888|     254|
|1800|     245|
|1663|     235|
|2266|     234|
|1352|     234|
| 483|     231|
| 348|     229|
|1730|     226|
|1985|     224|
|1941|     223|
|2233|     222|
|2142|     221|
|1431|     220|
+----+--------+
only showing top 20 rows

+----+---------+
|  id|outDegree|
+----+---------+
| 107|     1045|
|1684|      792|
|1912|      755|
|3437|      547|
|   0|      347|
|2543|      294|
|2347|      291|
|1888|      254|
|1800|      245|
|1663|      235|
|2266|      234|
|1352|      234|
| 483|      231|
| 348|      229|
|1730|      226|
|1985|      224|
|1941|      223|
|2233|      222|
|2142|      221|
|1431|      220|
+----+---------+
only showing top 20 rows



Now let's calculate pagerank to see who are the important ones in the network.

In [10]:
#[Your Code] to calculate pagerank on the graph and display nodes with top pageranks
results = fb_graph.pageRank(resetProbability=0.16, maxIter=10)

top_pageranks = results.vertices.orderBy(results.vertices.pagerank.desc()).limit(10)
top_pageranks.show()

+----+------------------+
|  id|          pagerank|
+----+------------------+
|3437| 30.70878054527504|
| 107|28.081776400814803|
|1684|25.699684286540457|
|   0|25.427815288933193|
|1912|15.679403098894708|
| 348| 9.493951584316306|
| 686| 8.950484478259174|
|3980| 8.743705001067275|
| 414|  7.30408236329689|
| 698| 5.317428593441851|
+----+------------------+



Shortest path is useful in many cases. Note that the `shortestPaths()` function in Grapgframes will actually calculate shortest distances (number of edges) from each node in the graph to all the nodes specified in `landmarks`. Here we want to calculate all the shortest paths from all users to two sample users with `id` of `0` and `25`, and then see the distribution of shortest distances from all users to them. So we first need to calculate shortest paths on the graph and extract the distance information.

In [11]:
#[Your Code] to calculate shortest paths from all nodes to node id 0 and 25
shortest_path = fb_graph.shortestPaths(landmarks=['0','25'])

Then we check the distribution of distances from all nodes to node 0 and 25.

In [12]:
#check the distribution of distances to node 0 and 25
shortest_path.select(fn.map_values('distances')[1].alias('distance')).groupBy('distance').count().orderBy('distance').show()
shortest_path.select(fn.map_values('distances')[0].alias('distance')).groupBy('distance').count().orderBy('distance').show()

+--------+-----+
|distance|count|
+--------+-----+
|       0|    1|
|       1|  347|
|       2| 1171|
|       3| 1742|
|       4|  519|
|       5|  117|
|       6|  142|
+--------+-----+

+--------+-----+
|distance|count|
+--------+-----+
|       0|    1|
|       1|   69|
|       2|  278|
|       3| 1171|
|       4| 1742|
|       5|  519|
|       6|  117|
|       7|  142|
+--------+-----+



Next we want to know the structure of this network, so we can get the clusters. We use label propagation to identify clusters, and show the number of clusters as well as size of clusters in the end.

In [13]:
#[Your Code] to use label propagation to identify clusters in the network; then show the total number of clusters you get and the size of each cluster.
clusters = fb_graph.labelPropagation(maxIter=5)
clusters.orderBy(fn.asc('label')).show()

+----+-----+
|  id|label|
+----+-----+
|1788|   53|
|1111|   53|
|1045|   53|
|1254|   53|
|1368|   53|
| 955|   53|
|1030|   53|
|1197|   53|
|1252|   53|
|1043|   53|
| 909|   53|
|1585|   59|
|1055|   59|
|1564|   59|
|1050|   59|
|1544|   59|
|1822|   59|
|1266|   59|
|1441|   59|
|1434|   59|
+----+-----+
only showing top 20 rows



In [14]:
clusters.select('label').distinct().count()

92

In [15]:
clusters.select('label').groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  558|    6|
|  442|   73|
| 2371|    8|
| 1447|    5|
| 2317|   20|
| 3125|  110|
| 1979|   15|
| 2775|    8|
| 2975|    1|
|  828|   17|
| 2021|    6|
| 2470|  156|
| 1073|    7|
| 2012|   28|
| 1501|   19|
| 2999|    5|
| 3295|   21|
| 1016|    7|
| 2715|    5|
| 1833|   73|
+-----+-----+
only showing top 20 rows



### Task 2 - Graph Analysis on Reddit Communities
We will work on a different graph dataset from Reddit in Task 2. Reddit is a large community for discussing different topics. In reddit, there are subreddits for specific topics. In particular, one community (subreddit) links to another community (subreddit) when a post refers to another post in another community. Therefore, the data here contains the posts from 2014 to 2017 that contain hyperlinks of another different subreddit. The data contains two parts, one is the hyperlinks in the body of reddit posts, the other is the hyperlinks in the title of reddit posts.

In [16]:
import pyspark.sql.functions as fn

#read data, two data files in total
reddit_link = spark.read.csv('./reddit_hyperlinks.csv.gz', header=True, inferSchema=True, sep=',')
reddit_link.show()
reddit_link_title = spark.read.csv('./reddit_hyperlinks_title.csv.gz', header=True, inferSchema=True, sep=',')
reddit_link_title.show()

+----------------+----------------+-------+----------------+--------------+
|SOURCE_SUBREDDIT|TARGET_SUBREDDIT|POST_ID|       TIMESTAMP|LINK_SENTIMENT|
+----------------+----------------+-------+----------------+--------------+
| leagueoflegends| teamredditteams|1u4nrps|12/31/2013 16:39|             1|
|      theredlion|          soccer| 1u4qkd|12/31/2013 18:18|            -1|
|    inlandempire|          bikela|1u4qlzs|  1/1/2014 14:54|             1|
|             nfl|             cfb|1u4sjvs|12/31/2013 17:37|             1|
|      playmygame|         gamedev|1u4w5ss|   1/1/2014 2:51|             1|
|      dogemarket|        dogecoin|1u4w7bs|12/31/2013 18:35|             1|
|     locationbot|     legaladvice|1u4wfes|  1/7/2014 20:17|             1|
|       indiefied|             aww|1u50pos|  3/3/2014 17:00|             1|
|    posthardcore|      bestof2013|1u5ccus|12/31/2013 23:16|             1|
|    posthardcore|        corejerk|1u5ccus|12/31/2013 23:16|             1|
|          g

Here we union the two dataframes first and then create the vertices/edges dataframes.

In [17]:
reddit_link_all = reddit_link.union(reddit_link_title)

In [18]:
#create vertices dataframe
reddit_vertices = reddit_link_all.select(fn.col('SOURCE_SUBREDDIT').alias('id')) \
                                 .union(reddit_link_all.select(fn.col('TARGET_SUBREDDIT').alias('id'))).distinct()
reddit_vertices.count()

67180

In [19]:
#create edges dataframe
reddit_edges = reddit_link_all.withColumnRenamed('SOURCE_SUBREDDIT', 'src').withColumnRenamed('TARGET_SUBREDDIT', 'dst')
reddit_edges.show()
reddit_edges.count()

+---------------+---------------+-------+----------------+--------------+
|            src|            dst|POST_ID|       TIMESTAMP|LINK_SENTIMENT|
+---------------+---------------+-------+----------------+--------------+
|leagueoflegends|teamredditteams|1u4nrps|12/31/2013 16:39|             1|
|     theredlion|         soccer| 1u4qkd|12/31/2013 18:18|            -1|
|   inlandempire|         bikela|1u4qlzs|  1/1/2014 14:54|             1|
|            nfl|            cfb|1u4sjvs|12/31/2013 17:37|             1|
|     playmygame|        gamedev|1u4w5ss|   1/1/2014 2:51|             1|
|     dogemarket|       dogecoin|1u4w7bs|12/31/2013 18:35|             1|
|    locationbot|    legaladvice|1u4wfes|  1/7/2014 20:17|             1|
|      indiefied|            aww|1u50pos|  3/3/2014 17:00|             1|
|   posthardcore|     bestof2013|1u5ccus|12/31/2013 23:16|             1|
|   posthardcore|       corejerk|1u5ccus|12/31/2013 23:16|             1|
|         gfycat|          india|1u5df

858488

Now build the graph with the two dataframes.

In [20]:
from graphframes import *
#build graph
reddit_graph = GraphFrame(reddit_vertices, reddit_edges)
print(reddit_graph)

GraphFrame(v:[id: string], e:[src: string, dst: string ... 3 more fields])


Let's start with degree centrality again. Here the importance of a community is better approximated by the links *to* the community (the posts in the community were referred in other communities), so we use inDegree centrality.

In [21]:
reddit_graph.inDegrees.sort(fn.desc("inDegree")).show()

+---------------+--------+
|             id|inDegree|
+---------------+--------+
|      askreddit|   26622|
|           iama|   13446|
|           pics|   12578|
|  todayilearned|   11124|
|          funny|   10777|
|         videos|   10013|
|      worldnews|    9944|
|           news|    7692|
|       politics|    6114|
|         gaming|    6097|
|  adviceanimals|    5503|
|            wtf|    5320|
|           gifs|    5214|
| writingprompts|    5056|
|leagueoflegends|    4856|
|        science|    4557|
|     the_donald|    4487|
| showerthoughts|    4202|
|        bitcoin|    4028|
|            nfl|    4000|
+---------------+--------+
only showing top 20 rows



Now let's use pagerank to determine the importance of community and show the top ones.

In [22]:
#[Your Code] to use pagerank to identify the most important communities based on the hyperlinks and display the top ones
results = reddit_graph.pageRank(resetProbability=0.16, maxIter=10)

top_pageranks = results.vertices.orderBy(results.vertices.pagerank.desc()).limit(10)
top_pageranks.show()

+-------------+------------------+
|           id|          pagerank|
+-------------+------------------+
|    askreddit|1458.0204619976894|
|         iama|1193.7232770052267|
|         pics| 748.4373997070086|
|        funny| 664.6783281635264|
|       videos| 619.5871312420129|
|todayilearned| 464.0774208095912|
|    worldnews| 418.1963576828606|
|       gaming|390.47308004670685|
|         news|  325.681508598367|
|      science| 295.4638461772499|
+-------------+------------------+



In the data, one column is the sentiment of the post with hyperlinks from one subreddit to another. So we can learn whether or not this is a positive post referring another subreddit. In other words, some posts might be negative when referring to the posts in other subreddits, implying that some communities may have conflicts. Can you identify which pairs of communities are more likely to have conflicts?

To do this, we can perform a query on the edges in the graph. Basically, we can obtain the average sentiment (`LINK_SENTIMENT` column) from one subreddit to another. To make sure this is not random, we should ONLY consider those pairs of communities with *at least 10 hyperlinks from one to another*.

In [24]:
#[Your Code] to identify the communities with significant conflicts
edges_filtered = reddit_graph.edges.groupby("src", "dst").agg(
    fn.count("LINK_SENTIMENT").alias("link_count"),
    fn.avg("LINK_SENTIMENT").alias("average_sentiment")
).filter("link_count >= 10")

# Step 2: (The average sentiment is already calculated in the step above)

# Step 3: Filter for negative sentiment (adjust the threshold as necessary)
conflict_pairs = edges_filtered.filter("average_sentiment < 0")

# Show the results
conflict_pairs.show()

+--------------------+--------------------+----------+--------------------+
|                 src|                 dst|link_count|   average_sentiment|
+--------------------+--------------------+----------+--------------------+
|         hearthstone|hearthstonecircle...|        39| -0.6410256410256411|
|      subredditdrama|             offbeat|        51| -0.0196078431372549|
|               drama|             worstof|        20|                -0.1|
|      subredditdrama|         liverpoolfc|        12|-0.16666666666666666|
|      subredditdrama|                dayz|        30|-0.06666666666666667|
|        circlebroke2|                 nfl|        27| -0.1111111111111111|
|               drama|       adviceanimals|        25|               -0.04|
|         circlebroke|          cringepics|        17|-0.05882352941176...|
|      subredditdrama| kitchenconfidential|        25|               -0.04|
|     trueredditdrama|      subredditdrama|        24|-0.08333333333333333|
|        cir

Next let's perform some searches on the graph. Assume you are a random walker in reddit communities - you just randomly browse posts without targeting any particular communities. Now assume you start your browsing trip in the `leagueoflegends` commuity (League of Legends is a Multiplayer Online Battle Arena (MOBA) e-sports video game). Now we are wondering whether (and in what way) you have a chance to reach other communities through the hyperlinks between communities. To do this, we can use breath-first search or motif finding.

Note that this is not likely to be a real action in practice and it is also not the actual role of those hyperlinks. We just use it as a simulated case of graph search. Now let's first see if you can reach `politics` community from `leagueoflegends` community directly.

In [25]:
paths = reddit_graph.bfs(fromExpr = "id = 'leagueoflegends'", toExpr = "id = 'politics'", maxPathLength = 1)
paths.show(truncate=False)

+---+
|id |
+---+
+---+



It seems no direct hyperlinks from `leagueoflegends` subreddit to `politics` subreddit. Therefore, we should check if there are shortest paths with length of 2 so that we may still reach `politics` community through another community. Can you identify those paths through `both` breath-first search and motif finding?

In [26]:
#[Your Code] to use breath-first search to find possible shortest paths from leagueoflegends to politics
paths_bfs = reddit_graph.bfs(
    fromExpr="id = 'leagueoflegends'", 
    toExpr="id = 'politics'", 
    maxPathLength=2
)
paths_bfs.show(truncate=False)

+-----------------+------------------------------------------------------+--------+----------------------------------------------+----------+
|from             |e0                                                    |v1      |e1                                            |to        |
+-----------------+------------------------------------------------------+--------+----------------------------------------------+----------+
|{leagueoflegends}|{leagueoflegends, greece, 47lojus, 2/25/2016 13:43, 1}|{greece}|{greece, politics, 56m7ofs, 10/9/2016 6:17, 1}|{politics}|
|{leagueoflegends}|{leagueoflegends, iama, 5ujbqxs, 2/16/2017 17:18, 1}  |{iama}  |{iama, politics, 1uideg, 1/5/2014 19:15, -1}  |{politics}|
|{leagueoflegends}|{leagueoflegends, iama, 4xsgn5s, 8/15/2016 0:18, 1}   |{iama}  |{iama, politics, 1uideg, 1/5/2014 19:15, -1}  |{politics}|
|{leagueoflegends}|{leagueoflegends, iama, 3cpug8s, 7/9/2015 14:29, 1}   |{iama}  |{iama, politics, 1uideg, 1/5/2014 19:15, -1}  |{politics}|
|{leag

In [27]:
#[Your Code] to use motif finding to find possible shortest paths from leagueoflegends to politics
motifs = reddit_graph.find("(a)-[ab]->(b); (b)-[bc]->(c)")
motifs = motifs.filter(
    "a.id = 'leagueoflegends' AND c.id = 'politics'"
)
motifs.show(truncate=False)

+-----------------+------------------------------------------------------+--------+----------------------------------------------+----------+
|a                |ab                                                    |b       |bc                                            |c         |
+-----------------+------------------------------------------------------+--------+----------------------------------------------+----------+
|{leagueoflegends}|{leagueoflegends, greece, 47lojus, 2/25/2016 13:43, 1}|{greece}|{greece, politics, 56m7ofs, 10/9/2016 6:17, 1}|{politics}|
|{leagueoflegends}|{leagueoflegends, iama, 5ujbqxs, 2/16/2017 17:18, 1}  |{iama}  |{iama, politics, 1uideg, 1/5/2014 19:15, -1}  |{politics}|
|{leagueoflegends}|{leagueoflegends, iama, 4xsgn5s, 8/15/2016 0:18, 1}   |{iama}  |{iama, politics, 1uideg, 1/5/2014 19:15, -1}  |{politics}|
|{leagueoflegends}|{leagueoflegends, iama, 3cpug8s, 7/9/2015 14:29, 1}   |{iama}  |{iama, politics, 1uideg, 1/5/2014 19:15, -1}  |{politics}|
|{leag

### Task 3 - Analyze Organizational Network
As the last task in all assignments, you will see no existing code and you will do a simple task on your own. 

The task is to read data from employees data (part of the data from official MySQL database samples). You need to read data from the `dept_emp.csv` file (employee department) and `titles.csv` file (employee titles).

In the data, there are columns `from_date` and `to_date`. For `to_date`, if it is `9999-01-01`, the employee is still at the company by the time of data collection (current employee). Therefore, we want to filter those employees with `to_date` as `9999-01-01` and with more than one records in the `dept_emp` table. That's what you need to obtain - you can use either dataframe operations or sql, and use `.show()` to display the results.

After that, we load the `titles.csv` file as a dataframe. In the dataframe, we can again use `to_date` column to filter all employees' current titles. Then with `emp_no`, you can join two dataframes and then obtain the distribution of titles for current employees who have worked at more than one department. Still, you can use either dataframe operations or sql, and use `.show()` to display the results.

In [31]:
#[Your Code] to complete task 3
dept_df = spark.read.csv('./dept_emp.csv', header=True, inferSchema=True, sep=',')
titles_df = spark.read.csv('./titles.csv', header=True, inferSchema=True, sep=',')

In [34]:
filtered_dept = dept_df.filter((fn.col('to_date') == '9999-01-01')) \
                        .groupBy('emp_no') \
                        .count() \
                        .filter(fn.col('count') >= 1)
filtered_dept.show()

+------+-----+
|emp_no|count|
+------+-----+
| 10206|    1|
| 10623|    1|
| 10817|    1|
| 11033|    1|
| 11141|    1|
| 11317|    1|
| 11458|    1|
| 11748|    1|
| 11858|    1|
| 12027|    1|
| 12799|    1|
| 12940|    1|
| 13289|    1|
| 13623|    1|
| 14570|    1|
| 14832|    1|
| 15447|    1|
| 15619|    1|
| 15727|    1|
| 15790|    1|
+------+-----+
only showing top 20 rows



In [35]:
filtered_titles = titles_df.filter(fn.col('to_date') == '9999-01-01')

In [36]:
full_df = filtered_dept.join(filtered_titles,'emp_no')

title_distribution = full_df.groupBy('title').count().orderBy('count',ascending=False)
title_distribution.show()

+------------------+-----+
|             title|count|
+------------------+-----+
|   Senior Engineer|85939|
|      Senior Staff|82024|
|          Engineer|30983|
|             Staff|25526|
|  Technique Leader|12055|
|Assistant Engineer| 3588|
|           Manager|    9|
+------------------+-----+

